<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase3/phase3_aboumahdi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/Thesis/phase-3/tokenized_phase3.json ./train.json

In [4]:
!pip install transformers

     |████████████████████████████████| 450kB 3.0MB/s 
     |████████████████████████████████| 870kB 50.6MB/s 
     |████████████████████████████████| 1.0MB 35.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=5f0474e5bd692f966d4dfb7688ad0d55f3a19dff95be25a5ff14aef49fd9137c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [13]:
import torch
from transformers import BertTokenizer, GPT2Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, json_file):
        """
        Args:
            json_file (string): Path to the json file with annotations.
        """
        self.dialogues = json.load(open(json_file))
        self.root_dir = json_file

        self.dialogues = sorted(self.dialogues, key=lambda x: len(x['history']))

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
      
        
        history_lst = self.dialogues[idx]['history']
        knowledge_lst = self.dialogues[idx]['knowledge']
        response_lst = self.dialogues[idx]['response']


        if(len(history_lst)>50):
          history_lst = history_lst[-50:]

        if(len(knowledge_lst)>50):
          knowledge_lst = knowledge_lst[:50]
        
        if(len(response_lst)>50):
          response_lst = response_lst[:50]
        

        history = torch.LongTensor(history_lst)
        knowledge = torch.LongTensor(knowledge_lst)
        response = torch.LongTensor(response_lst)

        sample = {'history': history, 'knowledge': knowledge, 'response': response}

        return sample

In [0]:
dataset = MyDataset('train.json')

In [11]:
print('len dataset is: {}'.format(len(dataset)))

len dataset is: 74092


In [0]:
def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_history = max([len(data['history']) for data in batch])
  max_len_knowledge_sample = max([len(data['knowledge']) for data in batch])
  max_len_response_sample = max([len(data['response']) for data in batch])
  
  padding_ind = 0 ## for bert is 0
  result_history = torch.zeros(len_batch, max_len_history)
  result_knowledge_sample = torch.zeros(len_batch, max_len_knowledge_sample)
  result_response_sample = torch.zeros(len_batch, max_len_response_sample)

  for i, data in enumerate(batch):
    p1 = len(data['history'])
    result_history[i, :p1] = data['history']
    p2 = len(data['knowledge'])
    result_knowledge_sample[i, :p2] = data['knowledge']
    p3 = len(data['response'])
    result_knowledge_sample[i, :p3] = data['response']



  return result_history.long(), result_knowledge_sample.long(), result_knowledge_sample.long()

sampler = torch.utils.data.SequentialSampler(dataset)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=128, sampler=sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [16]:
print("Ok let's test!!!!")
print(bert_tokenizer.decode(dataset[100]['history']))
print(bert_tokenizer.decode(dataset[100]['knowledge']))
print(gpt_tokenizer.decode(dataset[100]['response']))

Ok let's test!!!!
[CLS] hello [SEP]
[CLS] {'chosen _ steak _ 0':'a steak ( ) is a meat generally sliced across the muscle fibers, potentially including a bone.'} [SEP]
[CLS] Hi! I'm so hungry! I would so eat a steak right now!! I love meat [SEP]
